In [0]:
from pyspark.sql.functions import udf, col
from pyspark.sql import functions as F
import pandas as pd
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pyspark.sql.functions import col,to_date, date_format
from datetime import datetime
from datetime import timezone
from datetime import date, timedelta

pd.set_option('display.max_rows', 1500)


In [0]:
# %sql

# -- select min(dt) as MinDate, connectiondeviceid as Device from logfiles_dev group by connectiondeviceid ; 

# select split(dt,'T',-1)[0] as Date, count(dt) as NumRecords from logfiles_dev where connectiondeviceid = 'WW-005-TE' and dt between '2020-11-01' and '2020-11-30' group by Date;

Date,NumRecords
2020-11-29,86398
2020-11-19,86339
2020-11-15,86157
2020-11-23,86400
2020-11-24,86398
2020-11-17,85205
2020-11-27,86399
2020-11-16,85450
2020-11-25,86399
2020-11-20,86399


In [0]:
# df[df['dt']=='2020-11-15 00:00:33']

Out[54]:

,ConnectionDeviceID,dt,Pressure1,Pressure1_PSI,diff
21163,WW-005-TE,2020-11-15 00:00:33,1787,58.005495,0 days 00:00:01
47221,WW-005-TE,2020-11-15 00:00:33,1778,57.461538,0 days 00:00:01


In [0]:
# df_005_pd[df_005_pd['dt']>'2020-11-15T00:00:32'][['ConnectionDeviceID','dt']].head(3)

Out[62]:

,ConnectionDeviceID,dt
21163,WW-005-TE,2020-11-15T00:00:33
47221,WW-005-TE,2020-11-15T00:00:33.8
16352,WW-005-TE,2020-11-15T00:00:34.8


## Final Method > Single Device

In [0]:
# %sql
# -- Full Details of Missing Records Table

# CREATE TABLE default.Missing_Records_Details (
#   date STRING NOT NULL,
#   ID STRING NOT NULL,
#   hour STRING NOT NULL,
#   min STRING,
#   secs STRING)
# USING delta

In [0]:
# %sql
# -- total time for Missing Records Table

# CREATE TABLE default.total_time_Missing_Records (
#   Date STRING NOT NULL,
#   Device STRING NOT NULL,
#   total_time_minutes STRING NOT NULL)
# USING delta

In [0]:
# %sql

# CREATE TABLE default.long_Missing_Records (
#   ID STRING NOT NULL,
#   time_lapsed STRING NOT NULL,
#   date STRING,
#   time STRING)
# USING delta

In [0]:
# # -- %sql 

# from datetime import date, timedelta
# from pyspark.sql.types import StructType, StructField, StringType

# today = date.today()
# yesterday = today - timedelta(days = 2) 
# previous_date = yesterday.strftime('%Y-%m-%d')

# def missing_data(device,date=previous_date):
#   '''
#   Device ID - Provide the unique device ID whose usage needs to be investigated
#   Date - Provide the date in this format - yyyy-mm-dd
  
#   Returns: dataframe with all missing records, total minutes where no signal, DataFrame with Missing time instances where length > 10
    
#   '''
  
# #   for j, device in enumerate(device_list):
    
# #     print(device)
  
# #     #device = '\'' + device + '\''
# #     print(str(datetime.now()) + " Started Process for Device:" + device)
  
#   start_time = '00:00:00'
#   end_time = '23:59:59.8'
#   start_datetime = '(\'' + date + " " + start_time + '\')'
#   end_datetime = '(\'' + date + " " + end_time + '\')'

#   # Pressure1_PSI as p1, Pressure2_PSI as p2, Pressure3_PSI as p3
#   query = "select ConnectionDeviceID as ID, dt from logfiles_dev where dt > cast({} as timestamp) and dt < cast({} as timestamp) and ConnectionDeviceID in ('{}')".format(start_datetime, end_datetime, device)

#   df = spark.sql(query)

#   df_pd = df.toPandas().sort_values('dt')

#   #print(df_pd.ID.unique())

#   if (len(df_pd) > 1):

#     df_test = df_pd.copy(deep=True)
#     df_test['dt'] = pd.to_datetime(df_test['dt'])
#     df_test['dt'] = df_test['dt'].astype('datetime64[s]')

#     start_time1 = '00:00:00'
#     end_time1 = '23:59:59'
#     start_datetime1 = date + " " + start_time1
#     end_datetime1 = date + " " + end_time1

#     total_time = pd.date_range(start_datetime1,end_datetime1, freq="S")

#     df_time = pd.DataFrame(total_time, columns=['dt'])

#     df_time['ID'] = device

#     missing = df_time.merge(df_test, indicator=True, how='outer').query('_merge != "both"').drop('_merge', 1)

#     # Checking for time where no signal for more than 10 secs 
#     # 1) Total time - no signal - len of missing df /60 > gives minutes
#     # 2) Count of missing secs

#     total_noSignal = str(np.round((len(missing)/60),2)) + " minutes"
    
#     #total_time_Missing_Records

#     missing['dt'] = missing['dt'].astype(str)
#     missing[['date', 'time']] = missing.dt.str.split(" ", expand = True)
#     missing[['hour','min','secs']] = missing.time.str.split(":", expand = True)
    
#     # miss_recs df contains all the missing records details
    
#     miss_recs = missing.groupby(['date','ID','hour','min'],as_index=False).agg({'secs':'count'})
    
#     df_total_loss = pd.DataFrame()
    
#     df_total_loss['Date'] = miss_recs['date'].unique()
#     df_total_loss['Device'] = miss_recs['ID'].unique()
#     df_total_loss['total_time_minutes'] = total_noSignal
#     df_total_loss.drop_duplicates(keep='first', inplace=False)
    
#     # df_total_loss_spark contains entry for the device total loss in secs with no signal
    
#     df_total_loss_spark = spark.createDataFrame(df_total_loss)
    
#     #df_total_loss_spark.write.mode('append').insertInto('total_time_Missing_Records')

#     df_missing = miss_recs[miss_recs['secs'] > 10].copy()
#     df_missing.drop('date',axis=1,inplace=True)
    
    
#     if (len(df_missing) > 1):

#       df_missing['hour'] = df_missing['hour'].apply(str)
#       df_missing['min'] = df_missing['min'].apply(str)
#       df_missing['hr_min'] = df_missing[['hour','min']].apply(lambda x: ':'.join(x), axis = 1)

#       missing['hour'] = missing['hour'].apply(str)
#       missing['min'] = missing['min'].apply(str)
#       missing['hr_min'] = missing[['hour','min']].apply(lambda x: ':'.join(x), axis = 1)

#       missing_test = missing[ missing['hr_min'].apply(lambda title: df_missing['hr_min'].str.contains(title) ).any(1)]

#       df_f = pd.DataFrame()

#       for i in missing_test['hr_min'].unique():
#         df_f = df_f.append(missing_test[missing_test['hr_min']==i].iloc[0])

#       df_missing['secs'] = df_missing['secs'].apply(str)

#       df_missing.rename(columns ={'secs':'time_lapsed'},inplace=True)
      
#       # df_final > contains long missing records > 10 secs

#       df_final = pd.merge(df_missing,df_f[['date','hr_min','time']],on='hr_min',how='outer')

#       df_final.drop(['hour','min','hr_min'],axis=1,inplace=True)
      
#       df_miss_recs_spark = spark.createDataFrame(miss_recs)

#       df_final_spark = spark.createDataFrame(df_final)
      
#       return df_miss_recs_spark,df_total_loss_spark,df_final_spark

#     else:
#       column_names1 = "date|ID|hour|min|secs"
#       column_names2 = "ID|time_lapsed|date|time"
#       column_names3 = "Date|Device|total_time_minutes"
#       mySchema1 = StructType([StructField(c, StringType()) for c in column_names1.split("|")])
#       mySchema2 = StructType([StructField(c, StringType()) for c in column_names2.split("|")])
#       mySchema3 = StructType([StructField(c, StringType()) for c in column_names3.split("|")])
#       df_missing = spark.createDataFrame(data=[], schema=mySchema1)
#       df_long = spark.createDataFrame(data=[], schema=mySchema2)
#       df_total = spark.createDataFrame(data=[], schema=mySchema3)      
#       return(df_missing,df_total,df_long)

#   else:
#     return(0,0,'There are no records for this {} on the provided date {}'.format(device,date))

#   del miss_recs, total_noSignal , df_final



In [0]:
# df_miss_recs_spark.createOrReplaceTempView("Missing_Records_Details_View"),
# df_total_loss_spark.createOrReplaceTempView("total_time_Missing_Records_View"),
# df_final_spark.createOrReplaceTempView("long_Missing_Records_View")

In [0]:
# %sql
# --merge new incomplete data into log delta lake

# MERGE INTO total_time_Missing_Records
# USING total_time_Missing_Records_View
# ON (total_time_Missing_Records.Device = total_time_Missing_Records_View.Device and total_time_Missing_Records.Date = total_time_Missing_Records_View.Date and total_time_Missing_Records.total_time_minutes = total_time_Missing_Records_View.total_time_minutes )
# WHEN MATCHED THEN
#   UPDATE SET *
# WHEN NOT MATCHED
#   THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
0,0,0,0


In [0]:
# %sql
# --merge new incomplete data into log delta lake

# MERGE INTO Missing_Records_Details
# USING Missing_Records_Details_View
# ON (Missing_Records_Details.ID = Missing_Records_Details_View.ID and Missing_Records_Details.date = Missing_Records_Details_View.date and Missing_Records_Details.hour = Missing_Records_Details_View.hour and Missing_Records_Details.min = Missing_Records_Details_View.min and Missing_Records_Details.secs = Missing_Records_Details_View.secs )
# WHEN MATCHED THEN
#   UPDATE SET *
# WHEN NOT MATCHED
#   THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
417,0,0,417


In [0]:
# %sql
# --merge new incomplete data into log delta lake

# MERGE INTO long_Missing_Records
# USING long_Missing_Records_View
# ON (long_Missing_Records.ID = long_Missing_Records_View.ID and long_Missing_Records.date = long_Missing_Records_View.date and long_Missing_Records.time = long_Missing_Records_View.time and long_Missing_Records.time_lapsed = long_Missing_Records_View.time_lapsed  )
# WHEN MATCHED THEN
#   UPDATE SET *
# WHEN NOT MATCHED
#   THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
4,4,0,0


In [0]:
# %sql

# select split(dt,'T',-1)[0] as Date, count(dt) as NumRecords,connectiondeviceid as device from logfiles_dev where dt like '%2023-01-21%' group by Date,device ; -- dt between '2020-11-01' and '2020-11-30' group by Date; --connectiondeviceid = 'WW-005-TE' and

Date,NumRecords,device
2023-01-21,86397,WW-056-TE
2023-01-21,86399,WW-082-GF
2023-01-21,73230,WW-026-TE
2023-01-21,86400,WW-023-TE
2023-01-21,2603,WW-018-TE
2023-01-21,86157,WW-074-TE
2023-01-21,86220,WW-055-TE
2023-01-21,86095,WW-075-TE
2023-01-21,86400,WW-041-TE
2023-01-21,86470,WW-083-GF


## Setup for Multi Devices

### test method

In [0]:
# # -- %sql 

# from datetime import date, timedelta
# from pyspark.sql.types import StructType, StructField, StringType

# today = date.today()
# yesterday = today - timedelta(days = 2) 
# previous_date = yesterday.strftime('%Y-%m-%d')

# def missing_data_multi(device,date=previous_date):
#   '''
#   Device ID - Provide the unique device ID whose usage needs to be investigated
#   Date - Provide the date in this format - yyyy-mm-dd
  
#   Returns: dataframe with all missing records, total minutes where no signal, DataFrame with Missing time instances where length > 10
    
#   '''
  
# #   for j, device in enumerate(device_list):
    
# #     print(device)
  
# #     #device = '\'' + device + '\''
# #     print(str(datetime.now()) + " Started Process for Device:" + device)
  
#   start_time = '00:00:00'
#   end_time = '23:59:59.8'
#   start_datetime = '(\'' + date + " " + start_time + '\')'
#   end_datetime = '(\'' + date + " " + end_time + '\')'

#   # Pressure1_PSI as p1, Pressure2_PSI as p2, Pressure3_PSI as p3
#   query = "select ConnectionDeviceID as ID, dt from logfiles_dev where dt > cast({} as timestamp) and dt < cast({} as timestamp) and ConnectionDeviceID in ('{}')".format(start_datetime, end_datetime, device)

#   df = spark.sql(query)

#   df_pd = df.toPandas().sort_values('dt')

#   #print(df_pd.ID.unique())

#   if (len(df_pd) > 1):

#     df_test = df_pd.copy(deep=True)
#     df_test['dt'] = pd.to_datetime(df_test['dt'])
#     df_test['dt'] = df_test['dt'].astype('datetime64[s]')

#     start_time1 = '00:00:00'
#     end_time1 = '23:59:59'
#     start_datetime1 = date + " " + start_time1
#     end_datetime1 = date + " " + end_time1

#     total_time = pd.date_range(start_datetime1,end_datetime1, freq="S")

#     df_time = pd.DataFrame(total_time, columns=['dt'])

#     df_time['ID'] = device

#     missing = df_time.merge(df_test, indicator=True, how='outer').query('_merge != "both"').drop('_merge', 1)

#     # Checking for time where no signal for more than 10 secs 
#     # 1) Total time - no signal - len of missing df /60 > gives minutes
#     # 2) Count of missing secs

#     total_noSignal = str(np.round((len(missing)/60),2))
    
#     #total_time_Missing_Records

#     missing['dt'] = missing['dt'].astype(str)
#     missing[['date', 'time']] = missing.dt.str.split(" ", expand = True)
#     missing[['hour','min','secs']] = missing.time.str.split(":", expand = True)
    
#     # miss_recs df contains all the missing records details
    
#     miss_recs = missing.groupby(['date','ID','hour','min'],as_index=False).agg({'secs':'count'})
    
#     df_total_loss = pd.DataFrame()
    
#     df_total_loss['Date'] = miss_recs['date'].unique()
#     df_total_loss['Device'] = miss_recs['ID'].unique()
#     df_total_loss['total_time_minutes'] = total_noSignal
#     df_total_loss.drop_duplicates(keep='first', inplace=False)
    
#     # df_total_loss_spark contains entry for the device total loss in secs with no signal
    
#     #--df_total_loss_spark = spark.createDataFrame(df_total_loss)
    
#     #df_total_loss_spark.write.mode('append').insertInto('total_time_Missing_Records')

#     df_missing = miss_recs[miss_recs['secs'] > 10].copy()
#     df_missing.drop('date',axis=1,inplace=True)
    
    
#     if (len(df_missing) > 1):

#       df_missing['hour'] = df_missing['hour'].apply(str)
#       df_missing['min'] = df_missing['min'].apply(str)
#       df_missing['hr_min'] = df_missing[['hour','min']].apply(lambda x: ':'.join(x), axis = 1)

#       missing['hour'] = missing['hour'].apply(str)
#       missing['min'] = missing['min'].apply(str)
#       missing['hr_min'] = missing[['hour','min']].apply(lambda x: ':'.join(x), axis = 1)

#       missing_test = missing[ missing['hr_min'].apply(lambda title: df_missing['hr_min'].str.contains(title) ).any(1)]

#       df_f = pd.DataFrame()

#       for i in missing_test['hr_min'].unique():
#         df_f = df_f.append(missing_test[missing_test['hr_min']==i].iloc[0])

#       df_missing['secs'] = df_missing['secs'].apply(str)

#       df_missing.rename(columns ={'secs':'time_lapsed'},inplace=True)
      
#       # df_final > contains long missing records > 10 secs

#       df_final = pd.merge(df_missing,df_f[['date','hr_min','time']],on='hr_min',how='outer')

#       df_final.drop(['hour','min','hr_min'],axis=1,inplace=True)
      
#       #--df_miss_recs_spark = spark.createDataFrame(miss_recs)

#       #--df_final_spark = spark.createDataFrame(df_final)
      
# #       return df_miss_recs_spark,df_total_loss_spark,df_final_spark
#       return miss_recs,df_total_loss,df_final

#     else:
#         df_missing = pd.DataFrame(columns=['date','ID','hour','min','secs'])
#         df_long = pd.DataFrame(columns=['ID','time_lapsed','date','time'])
#         df_total = pd.DataFrame(columns=['Date','Device','total_time_minutes'])
# #         column_names1 = "date|ID|hour|min|secs"
# #         column_names2 = "ID|time_lapsed|date|time"
# #         column_names3 = "Date|Device|total_time_minutes"
# #         mySchema1 = StructType([StructField(c, StringType()) for c in column_names1.split("|")])
# #         mySchema2 = StructType([StructField(c, StringType()) for c in column_names2.split("|")])
# #         mySchema3 = StructType([StructField(c, StringType()) for c in column_names3.split("|")])
# #         df_missing = spark.createDataFrame(data=[], schema=mySchema1)
# #         df_long = spark.createDataFrame(data=[], schema=mySchema2)
# #         df_total = spark.createDataFrame(data=[], schema=mySchema3)      
# #         return df_missing,df_total,df_long
#         return df_missing,df_total,df_long
    
# #       column_names1 = "date|ID|hour|min|secs"
# #       column_names2 = "ID|time_lapsed|date|time"
# #       column_names3 = "Date|Device|total_time_minutes"
# #       mySchema1 = StructType([StructField(c, StringType()) for c in column_names1.split("|")])
# #       mySchema2 = StructType([StructField(c, StringType()) for c in column_names2.split("|")])
# #       mySchema3 = StructType([StructField(c, StringType()) for c in column_names3.split("|")])
# #       df_missing = spark.createDataFrame(data=[], schema=mySchema1)
# #       df_long = spark.createDataFrame(data=[], schema=mySchema2)
# #       df_total = spark.createDataFrame(data=[], schema=mySchema3)      
# #       return(df_missing,df_total,df_long)
    
#   else:
#     #return(0,0,'There are no records for this {} on the provided date {}'.format(device,date))
#     df_missing = pd.DataFrame(columns=['date','ID','hour','min','secs'])
#     df_long = pd.DataFrame(columns=['ID','time_lapsed','date','time'])
#     df_total = pd.DataFrame(columns=['Date','Device','total_time_minutes'])
#     return df_missing,df_total,df_long
# #       column_names1 = "date|ID|hour|min|secs"
# #       column_names2 = "ID|time_lapsed|date|time"
# #       column_names3 = "Date|Device|total_time_minutes"
# #       mySchema1 = StructType([StructField(c, StringType()) for c in column_names1.split("|")])
# #       mySchema2 = StructType([StructField(c, StringType()) for c in column_names2.split("|")])
# #       mySchema3 = StructType([StructField(c, StringType()) for c in column_names3.split("|")])
# #       df_missing = spark.createDataFrame(data=[], schema=mySchema1)
# #       df_long = spark.createDataFrame(data=[], schema=mySchema2)
# #       df_total = spark.createDataFrame(data=[], schema=mySchema3)      
# #       return df_missing,df_total,df_long

#   #del miss_recs, total_noSignal , df_final



### Final UDF for Multiple devices

In [0]:
from datetime import date, timedelta
from pyspark.sql.types import StructType, StructField, StringType

today = date.today()
yesterday = today - timedelta(days = 2) 
previous_date = yesterday.strftime('%Y-%m-%d')

def missing_data_multi(device,date=previous_date):
  '''
  Device ID - Provide the unique device ID whose usage needs to be investigated
  Date - Provide the date in this format - yyyy-mm-dd
  
  Returns: dataframe with all missing records, total minutes where no signal, DataFrame with Missing time instances where length > 10
    
  '''
  
  start_time = '00:00:00'
  end_time = '23:59:59.8'
  start_datetime = '(\'' + date + " " + start_time + '\')'
  end_datetime = '(\'' + date + " " + end_time + '\')'

  # Pressure1_PSI as p1, Pressure2_PSI as p2, Pressure3_PSI as p3
  query = "select ConnectionDeviceID as ID, dt from logfiles_dev where dt > cast({} as timestamp) and dt < cast({} as timestamp) and ConnectionDeviceID in ('{}')".format(start_datetime, end_datetime, device)

  df = spark.sql(query)

  df_pd = df.toPandas().sort_values('dt')

  if (len(df_pd) > 1):

    df_test = df_pd.copy(deep=True)
    df_test['dt'] = pd.to_datetime(df_test['dt'])
    df_test['dt'] = df_test['dt'].astype('datetime64[s]')

    start_time1 = '00:00:00'
    end_time1 = '23:59:59'
    start_datetime1 = date + " " + start_time1
    end_datetime1 = date + " " + end_time1

    total_time = pd.date_range(start_datetime1,end_datetime1, freq="S")

    df_time = pd.DataFrame(total_time, columns=['dt'])

    df_time['ID'] = device

    missing = df_time.merge(df_test, indicator=True, how='outer').query('_merge != "both"').drop('_merge', 1)

    # Checking for time where no signal for more than 10 secs 
    # 1) Total time - no signal - len of missing df /60 > gives minutes
    # 2) Count of missing secs

    total_noSignal = str(np.round((len(missing)/60),2))
    
    #total_time_Missing_Records

    missing['dt'] = missing['dt'].astype(str)
    missing[['date', 'time']] = missing.dt.str.split(" ", expand = True)
    missing[['hour','min','secs']] = missing.time.str.split(":", expand = True)
    
    miss_recs = missing.groupby(['date','ID','hour','min'],as_index=False).agg({'secs':'count'})
    
    df_total_loss = pd.DataFrame()
    
    df_total_loss['Date'] = miss_recs['date'].unique()
    df_total_loss['Device'] = miss_recs['ID'].unique()
    df_total_loss['total_time_minutes'] = total_noSignal
    df_total_loss.drop_duplicates(keep='first', inplace=False)
    df_missing = miss_recs[miss_recs['secs'] > 10].copy()
    df_missing.drop('date',axis=1,inplace=True)
    
    
    if (len(df_missing) > 1):

      df_missing['hour'] = df_missing['hour'].apply(str)
      df_missing['min'] = df_missing['min'].apply(str)
      df_missing['hr_min'] = df_missing[['hour','min']].apply(lambda x: ':'.join(x), axis = 1)

      missing['hour'] = missing['hour'].apply(str)
      missing['min'] = missing['min'].apply(str)
      missing['hr_min'] = missing[['hour','min']].apply(lambda x: ':'.join(x), axis = 1)

      missing_test = missing[ missing['hr_min'].apply(lambda title: df_missing['hr_min'].str.contains(title) ).any(1)]

      df_f = pd.DataFrame()

      for i in missing_test['hr_min'].unique():
        df_f = df_f.append(missing_test[missing_test['hr_min']==i].iloc[0])

      df_missing['secs'] = df_missing['secs'].apply(str)

      df_missing.rename(columns ={'secs':'time_lapsed'},inplace=True)
      
      # df_final > contains long missing records > 10 secs

      df_final = pd.merge(df_missing,df_f[['date','hr_min','time']],on='hr_min',how='outer')

      df_final.drop(['hour','min','hr_min'],axis=1,inplace=True)
      
      return miss_recs,df_total_loss,df_final

    else:
      df_missing = pd.DataFrame(columns=['date','ID','hour','min','secs'])
      df_long = pd.DataFrame(columns=['ID','time_lapsed','date','time'])
      df_total = pd.DataFrame(columns=['Date','Device','total_time_minutes'])
      return df_missing,df_total,df_long
   
  else:
    df_missing = pd.DataFrame(columns=['date','ID','hour','min','secs'])
    df_long = pd.DataFrame(columns=['ID','time_lapsed','date','time'])
    df_total = pd.DataFrame(columns=['Date','Device','total_time_minutes'])
    return df_missing,df_total,df_long


In [0]:
#devices = ['WW-024-TE']

units_ignore = pd.read_csv('/dbfs/FileStore/tables/Out_Cracking_Pressure/Units_Ignore.csv')
logfile_id_query = "select distinct ConnectionDeviceID as ID from logfiles_dev"
logfile_id_df = spark.sql(logfile_id_query)
device_list = (set(logfile_id_df.select('ID').toPandas()['ID'])) - (set(units_ignore['Unit']))
device_list = list(filter(None, device_list))

print(device_list)

missing_records = pd.DataFrame()
total_loss_time = pd.DataFrame()
long_missing_records = pd.DataFrame()
column_names1 = "date|ID|hour|min|secs"
column_names2 = "ID|time_lapsed|date|time"
column_names3 = "Date|Device|total_time_minutes"
mySchema1 = StructType([StructField(c, StringType()) for c in column_names1.split("|")])
mySchema2 = StructType([StructField(c, StringType()) for c in column_names2.split("|")])
mySchema3 = StructType([StructField(c, StringType()) for c in column_names3.split("|")])
df_miss_spark = spark.createDataFrame(data=[], schema=mySchema1)
final_spark = spark.createDataFrame(data=[], schema=mySchema2)
total_loss_spark = spark.createDataFrame(data=[], schema=mySchema3)


for i in device_list:
  df_miss_recs,df_total_loss,df_final = missing_data_multi(i)
  missing_records = missing_records.append(df_miss_recs)
  long_missing_records = long_missing_records.append(df_final)
  total_loss_time = total_loss_time.append(df_total_loss)
 
  del df_miss_recs,df_total_loss,df_final

  #print(total_loss_time.shape[0])

if total_loss_time.shape[0] > 0:
  df_miss_spark = spark.createDataFrame(missing_records)
  final_spark = spark.createDataFrame(long_missing_records)
  total_loss_spark = spark.createDataFrame(total_loss_time)
else:
  df_miss_spark,final_spark,total_loss_spark
  

['WW-066-TE', 'WW-023-TE', 'WW-083-GF', 'WW-089-GF', 'WW-006-GF', 'WW-079-GF', 'W2-201-GF', 'WW-085-GF', 'WW8', 'W2-999-GF', 'WW-024-TE', 'WW-033-TE', 'WW-045-TE', 'WW-005-TE', 'WW-078-GF', 'WW-043-TE', 'WW-042-TE', 'WW-021-TE', 'WW-041-TE', 'WW-066-TE_Testing', 'WW-044-TE', 'WW-075-TE', 'WW-039-TE', 'W2-202-GF', 'WW-082-GF', 'WW-022-GF', 'W2-203-GF', 'WW-081-GF', 'WW-067-GF', 'WW-074-TE', 'WW-035-TE', 'WW-026-TE', 'WW-071-TE', 'WW-032-TE', 'WW-100-GF']

In [0]:
# long_Missing_Records, total_time_Missing_Records , Missing_Records_Details

df_miss_spark.createOrReplaceTempView("Missing_Records_Details_View"),
total_loss_spark.createOrReplaceTempView("total_time_Missing_Records_View"),
final_spark.createOrReplaceTempView("long_Missing_Records_View")

In [0]:
%sql
--merge new incomplete data into log delta lake

MERGE INTO total_time_Missing_Records
USING total_time_Missing_Records_View
ON (total_time_Missing_Records.Device = total_time_Missing_Records_View.Device and total_time_Missing_Records.Date = total_time_Missing_Records_View.Date and total_time_Missing_Records.total_time_minutes = total_time_Missing_Records_View.total_time_minutes )
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED
  THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,0,0,10


In [0]:
# %sql

# select * from total_time_Missing_Records

Date,Device,total_time_minutes
2023-01-18,WW-023-TE,104.77
2023-01-18,WW-006-GF,101.27
2023-01-18,WW-078-GF,110.72
2023-01-18,WW-022-GF,207.23
2023-01-18,WW-071-TE,105.97
2023-01-21,WW-033-TE,245.78
2023-01-18,WW-083-GF,78.87
2023-01-18,WW-079-GF,82.48
2023-01-18,WW-024-TE,94.83
2023-01-18,WW-033-TE,303.3


In [0]:
%sql
--merge new incomplete data into log delta lake

MERGE INTO Missing_Records_Details
USING Missing_Records_Details_View
ON (Missing_Records_Details.ID = Missing_Records_Details_View.ID and Missing_Records_Details.date = Missing_Records_Details_View.date and Missing_Records_Details.hour = Missing_Records_Details_View.hour and Missing_Records_Details.min = Missing_Records_Details_View.min and Missing_Records_Details.secs = Missing_Records_Details_View.secs )
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED
  THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
5843,0,0,5843


In [0]:
%sql
--merge new incomplete data into log delta lake

MERGE INTO long_Missing_Records
USING long_Missing_Records_View
ON (long_Missing_Records.ID = long_Missing_Records_View.ID and long_Missing_Records.date = long_Missing_Records_View.date and long_Missing_Records.time = long_Missing_Records_View.time and long_Missing_Records.time_lapsed = long_Missing_Records_View.time_lapsed  )
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED
  THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1036,0,0,1036


In [0]:
%sql

select * from long_Missing_Records --where date='2023-01-18'

ID,time_lapsed,date,time
WW-071-TE,60,2023-01-18,14:01:00
WW-071-TE,60,2023-01-18,14:18:00
WW-071-TE,19,2023-01-18,14:25:17
WW-071-TE,60,2023-01-18,14:27:00
WW-071-TE,18,2023-01-18,17:03:18
WW-071-TE,60,2023-01-18,17:12:00
WW-071-TE,60,2023-01-18,18:06:00
WW-071-TE,11,2023-01-18,18:07:17
WW-071-TE,27,2023-01-18,18:24:04
WW-071-TE,12,2023-01-18,18:27:17


In [0]:
#missing_records[missing_records['ID']=='WW8'].sort_values('secs',ascending=False).head(20)

In [0]:
total_loss_time.sort_values('total_time_minutes',ascending=False)

Out[22]:

,Date,Device,total_time_minutes
0,2023-01-21,WW-075-TE,8.6
0,2023-01-21,WW-074-TE,7.43
0,2023-01-21,WW-022-GF,521.0
0,2023-01-21,WW-021-TE,28.45
0,2023-01-21,WW-078-GF,25.12
0,2023-01-21,WW-033-TE,245.78
0,2023-01-21,WW-026-TE,236.6
0,2023-01-21,WW-006-GF,17.85
0,2023-01-21,WW-071-TE,15.93
0,2023-01-21,WW-083-GF,12.05


In [0]:
long_missing_records[long_missing_records['ID']=='WW-083-GF']

Out[31]:

,ID,time_lapsed,date,time
0,WW-083-GF,12,2023-01-21,10:58:48
1,WW-083-GF,15,2023-01-21,11:07:12


In [0]:
missing_records[missing_records['ID']=='WW-083-GF'].sort_values('secs',ascending=False).head()

Out[34]:

,date,ID,hour,min,secs
228,2023-01-21,WW-083-GF,11,07,15
224,2023-01-21,WW-083-GF,10,58,12
449,2023-01-21,WW-083-GF,18,12,6
185,2023-01-21,WW-083-GF,10,07,4
579,2023-01-21,WW-083-GF,22,46,4


## Grouping long missing records to find the total length for continous loss

In [0]:
df_grp = final_spark.toPandas()

#hdf = final_spark.toHandy()

In [0]:
df_grp.head()

Out[30]:

,ID,time_lapsed,date,time
0,WW-083-GF,12,2023-01-21,10:58:48
1,WW-083-GF,15,2023-01-21,11:07:12
2,WW-006-GF,14,2023-01-21,00:06:40
3,WW-006-GF,12,2023-01-21,00:27:03
4,WW-006-GF,27,2023-01-21,00:38:00


In [0]:
df_test = df_grp[df_grp['ID']=='WW-026-TE']

In [0]:
# df_test.head()

Out[55]:

,ID,time_lapsed,date,time
809,WW-026-TE,60,2023-01-21,11:35:00
810,WW-026-TE,60,2023-01-21,11:40:00
811,WW-026-TE,16,2023-01-21,11:55:08
812,WW-026-TE,11,2023-01-21,12:04:43
813,WW-026-TE,60,2023-01-21,13:47:00


In [0]:
df_test['datetime'] = pd.to_datetime(df_test['date'] + ' ' + df_test['time'], format='%Y-%m-%d %H:%M:%S')

#df_time = df_test.set_index('time')
df_time = df_test.groupby([df_test.datetime.dt.hour,'ID']).first() #.reset_index()
df_time['Count'] = df_test.groupby([df_test.datetime.dt.hour,'ID']).size().values
df_time.rename(columns={'datetime':'Start_time'}, inplace=True)  #  .drop(['time_lapsed','date','time'],axis=1,inplace=True)
df_time.drop(['time_lapsed','date','time'],axis=1,inplace=True)
print(df_time)


<command-1759032063818540>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 df_test['datetime'] = pd.to_datetime(df_test['date'] + ' ' + df_test['time'], format='%Y-%m-%d %H:%M:%S')
 Start_time Count
datetime ID 
11 WW-026-TE 2023-01-21 11:35:00 3
12 WW-026-TE 2023-01-21 12:04:43 1
13 WW-026-TE 2023-01-21 13:47:00 1
15 WW-026-TE 2023-01-21 15:27:04 33
16 WW-026-TE 2023-01-21 16:00:00 60
17 WW-026-TE 2023-01-21 17:00:00 29
22 WW-026-TE 2023-01-21 22:12:05 48
23 WW-026-TE 2023-01-21 23:00:00 48

In [0]:
df_time.head()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1759032063818538> in <module> 
 ----> 1 df_time

 NameError : name 'df_time' is not defined